In [1]:
import json
import malaya

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
with open('filtered.json') as fopen:
    filtered = json.load(fopen)

with open('filtered_notin.json') as fopen:
    filtered_notin = json.load(fopen)
    
len(filtered), len(filtered_notin)

(1319596, 5973842)

In [3]:
# from tqdm import tqdm

# ind_filtered_notin, actual_filtered_notin = [], []
# for s in tqdm(filtered_notin):
#     if len(set(s.split()) & (ind_words)):
#         ind_filtered_notin.append(s)
#     else:
#         actual_filtered_notin.append(s)
        
# len(ind_filtered_notin), len(actual_filtered_notin)

In [4]:
import re
from unidecode import unidecode
import cleaning
from tqdm import tqdm

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [5]:
ind_filtered_notin = cleaning.multiprocessing(filtered, loop)

100%|██████████| 82474/82474 [00:01<00:00, 53573.51it/s]

100%|██████████| 82474/82474 [00:06<00:00, 13687.22it/s]


In [6]:
actual_filtered_notin = cleaning.multiprocessing(filtered_notin, loop)

100%|██████████| 373365/373365 [00:17<00:00, 21055.18it/s]


In [7]:
len(ind_filtered_notin), len(actual_filtered_notin)

(1319596, 5973842)

In [8]:
with open('english-words.json') as fopen:
    english_words = set(json.load(fopen))
    
with open('malays_word.json') as fopen:
    malays = set(json.load(fopen))

In [9]:
# with open('counts_1grams-second.txt') as fopen:
#     count = list(filter(None, fopen.read().split('\n')))
    
# with open('counts_1grams.txt') as fopen:
#     count.extend(list(filter(None, fopen.read().split('\n'))))
    
# count = set([c.split('\t')[0] for c in count])
# len(count)

In [10]:
english_words = malaya.texts._english_words._english_words | english_words
len(english_words)

272249

In [11]:
english_minus = english_words - malays
len(english_words)

272249

In [12]:
english_minus = {i for i in english_minus if 'haha' not in i and i != 'rt' and 'yeay' not in i and\
                'yes' not in i and 'ooo' not in i and 'insha' not in i and 'huhu' not in i and\
                'insya' not in i and 'hew' not in i and 'uwuu' not in i and\
                 'meow' not in i and 'aiii' not in i and 'alham' not in i and 'mashaa' not in i\
                 and i not in ['takda', 'cer']}

len(english_minus)

131801

In [13]:
rojak, malays = [], []
for s in tqdm(actual_filtered_notin):
    if len(set(s.split()) & english_minus):
        rojak.append(s)
    else:
        malays.append(s)

100%|██████████| 5973842/5973842 [00:19<00:00, 313461.10it/s]


In [14]:
len(rojak), len(malays)

(711454, 5262388)

In [15]:
rojak[:10]

['dia dah tua put dah nak plus dia tak start regularly kat man utd so mesti ka',
 'guys tolong rt tweet ni sampai owner dia dapat phone ni tertinggal kat belakang teksi pakcik saya model oppo r s',
 'jenis jenis orang stalking di media sosial pakai akun palsu pakai akun temannya sanak saudaranya handai tau',
 'ajax spurs lah anti menstrim',
 'lia pulang mereka semuanya pedo kecuali aku jangan mau',
 'nice igstory harini dah tak nmpak org repost sudan meal project tu',
 'beomgyu ngambilin confetti yang nyangkut di rambut jimin dong liat gini aja soft akutuh cha',
 'bts at rose bowl day jadi mulai hari ini aku akan berusaha menjadi dokter strange karena cincin ini',
 'bukan sengaja nak samakan tapi tu laaa dah ter sama kan obvious okayyy',
 'bolehhhh hehehe']

In [16]:
malays[:10]

['',
 'memiliki sedikit iman lebih berharga dari pada memiliki segudang emas',
 'dom jakpus sih bebas mau ketemuan or shopee',
 'on jadahnyaaa in sorry bad english hihuheheho',
 'sis tak faham apa yang mungkin ini puncanya tu',
 'martabak terang bulan martabak untuk yg asin gurih a k a martabak telor terang bulan untuk yg manis yg gw s',
 'sejarah susah',
 'loop in nama dlm email pon boleh jd issue dah org email aku reply all jelaaa ade mase pulak aku nak tengok satu nama recipients',
 'tak sakit pun tapi saja nak bau minyak freshcare sbb bau lavender',
 'rosmah']

In [17]:
ind_filtered_notin[:10]

['ternyata kl lg sdih bisa ngasilin makanan enak',
 'abu kampret',
 'bapa saya suka pake oppo saya sukanya nokia kaka saya sukanya samsung yg penting punya hape aja',
 'ngelamar kasih cincin tp kok mukanya songong ya sedih gue liatnya',
 'caption iki nggarai uwong males nikah min ya kali manusia arep punah ngunu neg gak nikah',
 'pertanyaannya sederhana jika kami memang dukung prabowo ngapain selama kampanye kemarin capek dukung jokowi sampa',
 'untuk mengamankan suara partai ahmad rofiq selaku sekjen partai perindo meminta kepada seluruh caleg dan struktur',
 'bisa dapet duit ini kaga punya mobil juga kan kaya gemer gemer ini kaga',
 'valentino rossi tidak setuju kompetisi motogp dimulai dari eropa',
 'tidur di ubin biar ga jatoh lg']

In [18]:
with open('social-language.json', 'w') as fopen:
    json.dump({'rojak': rojak,
              'malay': malays,
              'ind': ind_filtered_notin}, fopen)